In [1]:
from copy import deepcopy
from pathlib import Path
import cv2
import matplotlib.pylab as plt
import numpy as np
from scipy.io import loadmat
from scipy.ndimage import gaussian_filter
from tqdm import tqdm
import cv2
import pandas as pd
import numpy as np 
import cv2 
from matplotlib import pyplot as plt 
import math


In [2]:
# Read the bounding box coordinates from the CSV file
csv_path_1 = '129_1_clean.txt'
df1_left_view = pd.read_csv(csv_path_1)

# Read the bounding box coordinates from the CSV file
csv_path_2 = '129_2_clean.txt'
df2_right_view = pd.read_csv(csv_path_2)

In [63]:
display(df1_left_view)
display(df2_right_view)

,frame,id,x,y,x_offset,y_offset
0,1,1,1212.98,416.74,26.04,14.37
1,1,2,1151.14,415.00,27.58,10.34
2,1,3,1487.32,431.94,17.38,19.86
3,1,4,1080.11,337.38,18.68,6.46
4,1,5,1220.90,325.30,21.28,10.54
...,...,...,...,...,...,...
30196,3117,5,1011.60,235.99,10.38,22.01
30197,3117,6,1304.99,217.40,15.01,24.80
30198,3117,7,969.60,489.40,23.90,16.40
30199,3117,8,1037.36,245.93,14.89,29.79


,frame,id,x,y,x_offset,y_offset
0,1,1,1010.39,357.55,21.50,7.74
1,1,2,1151.46,337.71,20.11,17.63
2,1,3,1173.87,341.47,11.20,12.02
3,1,4,1446.30,460.60,19.11,20.90
4,1,5,1066.87,626.29,48.07,6.87
...,...,...,...,...,...,...
30644,3118,5,949.07,696.08,23.25,33.30
30645,3118,6,1067.44,288.15,38.96,21.85
30646,3118,7,1188.06,193.33,33.94,17.14
30647,3118,8,883.91,514.11,25.29,14.69


In [4]:
df1_left_frame789 = df1_left_view[df1_left_view['frame'] == 789]
df2_right_frame789 = df2_right_view[df2_right_view['frame'] == 789]

display(df1_left_frame789)
display(df2_right_frame789)

,frame,id,x,y,x_offset,y_offset
6620,789,1,1149.29,443.51,19.76,23.76
6621,789,2,1118.95,386.06,17.31,36.58
6622,789,4,1017.20,315.56,8.60,22.30
6623,789,5,1193.10,303.26,21.76,8.07
6624,789,6,1288.33,365.61,11.67,14.32
6625,789,7,1129.56,445.91,19.12,14.84
6626,789,8,1038.59,355.62,18.25,26.72
6627,789,9,1178.52,532.07,23.40,36.92


,frame,id,x,y,x_offset,y_offset
6638,789,1,940.86,336.07,12.60,22.22
6639,789,2,1220.70,388.78,20.08,14.30
6640,789,3,1133.53,319.53,25.75,12.19
6641,789,5,1066.78,569.13,22.93,23.90
6642,789,6,1035.73,469.18,20.97,24.87
6643,789,7,1007.64,417.85,19.82,32.39
6644,789,8,1048.95,469.40,21.45,17.68
6645,789,9,917.87,379.61,20.61,31.62


In [5]:
CameraDataFile = "stereoParams_Dep4.mat"
view1 = 'right'
ImagePath1 = 'frame_800_without_boxes_left.jpg'
view2 = 'left'
ImagePath2 = 'frame_800_without_boxes_right.jpg'

In [6]:
def Undistorter(ImagePath,view,CameraDataFile):
    
    dd = loadmat(CameraDataFile)
    distCoeffs1 = deepcopy(dd["distortionCoefficients1"])
    distCoeffs2 = deepcopy(dd["distortionCoefficients2"])
    cameraMatrix1 = deepcopy(dd["intrinsicMatrix1"])
    cameraMatrix2 = deepcopy(dd["intrinsicMatrix2"])
    R = deepcopy(dd["rotationOfCamera2"])
    T = deepcopy(dd["translationOfCamera2"])
    cameraMatrix1[0:2, 2] += 1
    cameraMatrix2[0:2, 2] += 1
    img = cv2.imread(ImagePath)
    img_undistorted = cv2.undistort(img, cameraMatrix1, distCoeffs1)
    filename = view + '_undistortedV2.jpg'
    cv2.imwrite(filename, img_undistorted)
    
Undistorter(ImagePath1,view1,CameraDataFile)
Undistorter(ImagePath2,view2,CameraDataFile)

In [7]:
LeftImagePath = 'left_undistorted.jpg'
RightImagePath = 'right_undistorted.jpg'

In [8]:
def EpipolarGeometry(CameraDataFile,LeftImagePath,RightImagePath):
    
    dd = loadmat(CameraDataFile)
    distCoeffs1 = deepcopy(dd["distortionCoefficients1"])
    distCoeffs2 = deepcopy(dd["distortionCoefficients2"])
    cameraMatrix1 = deepcopy(dd["intrinsicMatrix1"])
    cameraMatrix2 = deepcopy(dd["intrinsicMatrix2"])
    R = deepcopy(dd["rotationOfCamera2"])
    T = deepcopy(dd["translationOfCamera2"])
    cameraMatrix1[0:2, 2] += 1
    cameraMatrix2[0:2, 2] += 1
    # Projection matrices
    P1 = np.dot(cameraMatrix1, np.hstack((np.eye(3), np.zeros((3, 1)))))
    P2 = np.dot(cameraMatrix2, np.hstack((R, T.reshape(3, 1))))
    
    # Load the undistorted stereo images
    img_left = cv2.imread(LeftImagePath, cv2.IMREAD_GRAYSCALE)
    img_right = cv2.imread(RightImagePath, cv2.IMREAD_GRAYSCALE)

    # Initialize SIFT detector
    sift = cv2.SIFT_create()

    # Detect keypoints and compute descriptors
    keypoints_left, descriptors_left = sift.detectAndCompute(img_left, None)
    keypoints_right, descriptors_right = sift.detectAndCompute(img_right, None)

    # Undistort keypoints
    keypoints_left_undistorted = cv2.undistortPoints(np.array([kp.pt for kp in keypoints_left]).reshape(-1, 1, 2), cameraMatrix1, distCoeffs1, P=cameraMatrix1)
    keypoints_right_undistorted = cv2.undistortPoints(np.array([kp.pt for kp in keypoints_right]).reshape(-1, 1, 2), cameraMatrix2, distCoeffs2, P=cameraMatrix2)

    # Convert keypoints back to OpenCV KeyPoint format
    keypoints_left_undistorted = [cv2.KeyPoint(x[0][0], x[0][1], 1) for x in keypoints_left_undistorted]
    keypoints_right_undistorted = [cv2.KeyPoint(x[0][0], x[0][1], 1) for x in keypoints_right_undistorted]

    # Create FLANN matcher object
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Match descriptors of left and right images
    matches = flann.knnMatch(descriptors_left, descriptors_right, k=2)

    # Apply ratio test
    good_matches = []
    pts_left = []
    pts_right = []

    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good_matches.append(m)
            pts_left.append(keypoints_left_undistorted[m.queryIdx].pt)
            pts_right.append(keypoints_right_undistorted[m.trainIdx].pt)

    pts_left = np.int32(pts_left) 
    pts_right = np.int32(pts_right) 
    F, mask = cv2.findFundamentalMat(pts_left, 
                                     pts_right, 
                                     cv2.FM_LMEDS) 

    # We select only inlier points 
    pts_left = pts_left[mask.ravel() == 1] 
    pts_right = pts_right[mask.ravel() == 1] 


    return (F,keypoints_left_undistorted,keypoints_right_undistorted)

In [9]:
Epilor_Details = EpipolarGeometry(CameraDataFile,LeftImagePath,RightImagePath)
# print(Epilor_Details)

In [103]:
def find_matching_points(left_point, fundamental_matrix, right_image_keypoints):
    
    # Convert left point to homogeneous coordinates
    left_point = np.array([left_point[0], left_point[1], 1])

    # Compute the epipolar line in the right image corresponding to the left point
    epipolar_line = np.dot(fundamental_matrix, left_point)
    # Calculate the bounds for the search area in the right image
    search_area_width = 100  # You can adjust this based on your requirement
    search_area_height = 100  # You can adjust this based on your requirement

    # Define the search area
    search_area = ((left_point[0] - search_area_width, left_point[1] - search_area_height),
                   (left_point[0] + search_area_width, left_point[1] + search_area_height))

    # Initialize a list to store potential matching points
    potential_matching_points = []

    # Iterate through all keypoints in the right image
    for keypoint in right_image_keypoints:
        x, y = keypoint.pt

        # Check if the keypoint lies within the search area
        if search_area[0][0] <= x <= search_area[1][0] and search_area[0][1] <= y <= search_area[1][1]:
            # Compute the distance between the keypoint and the epipolar line
            distance = abs(epipolar_line[0] * x + epipolar_line[1] * y + epipolar_line[2]) / \
                       np.sqrt(epipolar_line[0] ** 2 + epipolar_line[1] ** 2)

            # You can adjust this threshold based on your requirement
            if distance < 30:  # Adjust this threshold as needed
                potential_matching_points.append((x, y))

    return potential_matching_points

In [104]:
def find_matching_points_reverse(right_point, fundamental_matrix, left_image_keypoints):
    
    # Convert left point to homogeneous coordinates
    right_point = np.array([right_point[0], right_point[1], 1])

    # Compute the epipolar line in the right image corresponding to the left point
    epipolar_line = np.dot(fundamental_matrix, right_point)
    # Calculate the bounds for the search area in the right image
    search_area_width = 100  # You can adjust this based on your requirement
    search_area_height = 100  # You can adjust this based on your requirement

    # Define the search area
    search_area = ((right_point[0] - search_area_width, right_point[1] - search_area_height),
                   (right_point[0] + search_area_width, right_point[1] + search_area_height))

    # Initialize a list to store potential matching points
    potential_matching_points = []

    # Iterate through all keypoints in the right image
    for keypoint in left_image_keypoints:
        x, y = keypoint.pt

        # Check if the keypoint lies within the search area
        if search_area[0][0] <= x <= search_area[1][0] and search_area[0][1] <= y <= search_area[1][1]:
            # Compute the distance between the keypoint and the epipolar line
            distance = abs(epipolar_line[0] * x + epipolar_line[1] * y + epipolar_line[2]) / \
                       np.sqrt(epipolar_line[0] ** 2 + epipolar_line[1] ** 2)

            # You can adjust this threshold based on your requirement
            if distance < 30:  # Adjust this threshold as needed
                potential_matching_points.append((x, y))

    return potential_matching_points

In [105]:
def closest_tuple(given_tuple, tuple_list):
    
    closest = None
    min_distance = float('inf')
    
    for tpl in tuple_list:
        distance = math.sqrt(sum((x - y) ** 2 for x, y in zip(given_tuple, tpl)))
        if distance < min_distance:
            min_distance = distance
            closest = tpl
    
    return closest

In [106]:
def tuple_differences(tuple1, tuple2):
    if len(tuple1) != len(tuple2):
        raise ValueError("Tuples must have the same length")
    
    return tuple(x - y for x, y in zip(tuple1, tuple2))

In [107]:
def Find_Matching_ID(left_id, left_point_tuple, right_view_dataframe, fundamental_matrix, right_image_keypoints):
    
    matching_points = find_matching_points(left_point_tuple, fundamental_matrix, right_image_keypoints)
    
    options = []
    
    df = pd.DataFrame(columns=['left_id','right_id','difference','frame'])
    difference = None
    for index, row in right_view_dataframe.iterrows():
        tuple_right = (row['x'],row['y'])
        
        difference = abs(tuple_differences(tuple_right,closest_tuple(tuple_right, matching_points))[0]) + abs(tuple_differences(tuple_right,closest_tuple(tuple_right, matching_points))[1])
        new_row_data = {'left_id': left_id, 'right_id': row['id'],'difference': difference,'frame':row['frame']}
        new_row = pd.DataFrame(new_row_data,index=[0])
        df = pd.concat([df, new_row], ignore_index=True)

        
    return df

In [108]:
def Find_Matching_ID_reverse(right_id, right_point_tuple, left_view_dataframe, fundamental_matrix, left_image_keypoints):
    
    matching_points = find_matching_points_reverse(right_point_tuple, fundamental_matrix, left_image_keypoints)
    
    options = []
    
    df = pd.DataFrame(columns=['right_id','left_id','difference','frame'])
    difference = None
    for index, row in left_view_dataframe.iterrows():
        tuple_left = (row['x'],row['y'])
        
        difference = abs(tuple_differences(tuple_left,closest_tuple(tuple_left, matching_points))[0]) + abs(tuple_differences(tuple_left,closest_tuple(tuple_left, matching_points))[1])
        new_row_data = {'right_id': right_id, 'left_id': row['id'],'difference': difference,'frame':row['frame']}
        new_row = pd.DataFrame(new_row_data,index=[0])
        df = pd.concat([df, new_row], ignore_index=True)

        
    return df

In [109]:
fundamental_matrix = Epilor_Details[0]
left_id = 6
left_point_tuple = (1287.82,366.35)
right_view_dataframe = df2_right_view
right_image_keypoints = Epilor_Details[2]
# print(fundamental_matrix)
# id 1: 940.86, 336.07

# 8	1037.36, 245.93 left 
# 9 right 917.87, 379.61


# 2	1118.95	386.06	 left
# 7	1007.64	417.85 right 

# id 9 left: 1180.07	535.37
# id 5 right: 1066.78	569.13

# id 6 left: 1287.82	366.35
# id 2 right: 1220.70	388.78

display(df2_right_view)

df_mathces = Find_Matching_ID(left_id, left_point_tuple, df2_right_view, fundamental_matrix, right_image_keypoints)

,frame,id,x,y,x_offset,y_offset
0,1,1,1010.39,357.55,21.50,7.74
1,1,2,1151.46,337.71,20.11,17.63
2,1,3,1173.87,341.47,11.20,12.02
3,1,4,1446.30,460.60,19.11,20.90
4,1,5,1066.87,626.29,48.07,6.87
...,...,...,...,...,...,...
30644,3118,5,949.07,696.08,23.25,33.30
30645,3118,6,1067.44,288.15,38.96,21.85
30646,3118,7,1188.06,193.33,33.94,17.14
30647,3118,8,883.91,514.11,25.29,14.69


In [110]:
fundamental_matrix = Epilor_Details[0]
right_id = 2
right_point_tuple = (1220.70,388.78)
left_view_dataframe = df1_left_view
left_image_keypoints = Epilor_Details[1]
# print(fundamental_matrix)
# id 1: 940.86, 336.07

# 8	1037.36, 245.93 left 
# 9 right 917.87, 379.61

# 2	1118.95	386.06	 left
# 7	1007.64	417.85 right 

# id 9 left: 1180.07	535.37
# id 5 right: 1066.78	569.13

# id 6 left: 1287.82	366.35
# id 2 right: 1220.70	388.78

display(df1_left_view)

df_mathces_reverse = Find_Matching_ID_reverse(right_id, right_point_tuple, df1_left_view, fundamental_matrix, left_image_keypoints)

,frame,id,x,y,x_offset,y_offset
0,1,1,1212.98,416.74,26.04,14.37
1,1,2,1151.14,415.00,27.58,10.34
2,1,3,1487.32,431.94,17.38,19.86
3,1,4,1080.11,337.38,18.68,6.46
4,1,5,1220.90,325.30,21.28,10.54
...,...,...,...,...,...,...
30196,3117,5,1011.60,235.99,10.38,22.01
30197,3117,6,1304.99,217.40,15.01,24.80
30198,3117,7,969.60,489.40,23.90,16.40
30199,3117,8,1037.36,245.93,14.89,29.79


In [111]:
def lowest_avg_right_id(average_difference_per_id):
    lowest_avg_id = average_difference_per_id.idxmin()
    return lowest_avg_id

def get_best_match(df_mathces):
    # Group by 'frame' and find the index of the row with the minimum 'difference' value in each group
    min_indices = df_mathces.groupby('frame')['difference'].idxmin()
    
    # Extract the corresponding rows using the indices
    min_diff_rows = df_mathces.loc[min_indices]
    
    # Find the three most common right_id values
    top_3_common_ids = min_diff_rows['right_id'].value_counts().head(3).index.tolist()
    
    # Filter the DataFrame to include only rows with the top 3 common right_id values
    filtered_rows = min_diff_rows[min_diff_rows['right_id'].isin(top_3_common_ids)]

    # Calculate the average difference value per right_id for the top 3 common right_id values
    average_difference_per_id = filtered_rows.groupby('right_id')['difference'].mean()
    
    df_result = average_difference_per_id.to_frame().reset_index()
        
    return df_result


In [112]:
def lowest_avg_left_id(average_difference_per_id):
    lowest_avg_id = average_difference_per_id.idxmin()
    return lowest_avg_id

def get_best_match_reverse(df_mathces):
    # Group by 'frame' and find the index of the row with the minimum 'difference' value in each group
    min_indices = df_mathces.groupby('frame')['difference'].idxmin()
    
    # Extract the corresponding rows using the indices
    min_diff_rows = df_mathces.loc[min_indices]
    
    # Find the three most common right_id values
    top_3_common_ids = min_diff_rows['left_id'].value_counts().head(3).index.tolist()
    
    # Filter the DataFrame to include only rows with the top 3 common right_id values
    filtered_rows = min_diff_rows[min_diff_rows['left_id'].isin(top_3_common_ids)]

    # Calculate the average difference value per right_id for the top 3 common right_id values
    average_difference_per_id = filtered_rows.groupby('left_id')['difference'].mean()
    
    lowest_id = lowest_avg_left_id(average_difference_per_id)
        
    df_result = average_difference_per_id.to_frame().reset_index()
        
    return df_result


In [131]:
predictions_right_using_left = get_best_match(df_mathces)
predictions_left_using_right = get_best_match_reverse(df_mathces_reverse)

predictions_right_using_left['right_id'] = predictions_right_using_left['right_id'].astype(int)
predictions_left_using_right['left_id'] = predictions_left_using_right['left_id'].astype(int)

In [132]:
print(left_id)
display(predictions_right_using_left)
print(right_id)
display(predictions_left_using_right)

6


,right_id,difference
0,2,33.679079
1,7,60.234211
2,12,105.442958


2


,left_id,difference
0,1,25.222071
1,6,30.845701
2,12,56.714496


In [137]:
for x in predictions_left_using_right['left_id']:
    print(x)

1
6
12


In [142]:
# Given tuple
tuple_value = (6, 2)

# Check if the first value is in the index of dataframe1
if 6 in list(predictions_left_using_right['left_id']):
    # Check if the second value is in the index of dataframe2
    if tuple_value[1] in list(predictions_right_using_left['right_id']):
        print("match")

match
